In [ ]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import NASNetMobile, MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import load_model
from joblib import Parallel, delayed
from functools import partial
from os import listdir 
import time
import pickle 
import numpy as np
from tqdm import tqdm
from scipy import sparse

In [15]:
DP_DIR = "../../data/"
BASE_SIZE = 256
NCATS = 340

In [ ]:
def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

def get_input(df, size, lw, time_color):
    df['drawing'] = df['drawing'].apply(ast.literal_eval)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw,
                                 time_color=time_color)
    x = preprocess_input(x).astype(np.float32)
    y = keras.utils.to_categorical(df.y, num_classes=NCATS)
    return x, y
    
def image_generator_xd(size, batchsize, ks, lw=6, time_color=True):
    partial_get_input = partial(get_input, size=size, lw=lw, time_color=time_color)
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            tmp_chunks = []
            for chunk in pd.read_csv(filename, chunksize=batchsize, nrows=batchsize*8):
                tmp_chunks.append(chunk)
                if len(tmp_chunks) == 8:
                    inputs = Parallel(n_jobs=1)(delayed(partial_get_input)(chunk) for chunk in tmp_chunks)
                    tmp_chunks = [] 
                    for inp in inputs:
                        yield inp
            inputs = Parallel(n_jobs=1)(delayed(partial_get_input)(chunk) for chunk in tmp_chunks)
            for inp in inputs:
                yield inp




In [ ]:
train_datagen = image_generator_xd(size=128, batchsize=10000, ks=99)

In [66]:
start = time.time()
for i in tqdm(range(100)):
    a = next(train_datagen)
    with open("partition_"+str(i)+".npy", 'wb') as of:
        pickle.dump(a, of)
        

print(time.time()-start)


100%|██████████| 100/100 [10:08<00:00,  4.40s/it]


608.15407371521


In [14]:
start = time.time()
get_input(df[:1000], 128, 6, False)
print(time.time()-start)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(array([[[[-1.        ],
          [-1.        ],
          [-1.        ],
          ...,
          [-1.        ],
          [-1.        ],
          [-1.        ]],
 
         [[-1.        ],
          [-0.49803922],
          [ 1.        ],
          ...,
          [-1.        ],
          [-1.        ],
          [-1.        ]],
 
         [[-1.        ],
          [ 1.        ],
          [ 1.        ],
          ...,
          [-1.        ],
          [-1.        ],
          [-1.        ]],
 
         ...,
 
         [[-1.        ],
          [-1.        ],
          [-1.        ],
          ...,
          [-1.        ],
          [-1.        ],
          [-1.        ]],
 
         [[-1.        ],
          [-1.        ],
          [-1.        ],
          ...,
          [-1.        ],
          [-1.        ],
          [-1.        ]],
 
         [[-1.        ],
          [-1.        ],
          [-1.        ],
          ...,
          [-1.        ],
          [-1.        ],
    

0.4958078861236572


In [ ]:
50/2

In [ ]:
with open("test_file", 'wb') as of:
    pickle.dump(df, of)
    #np.save(of, df, allow_pickle=True, fix_imports=True)

In [ ]:
start = time.time()
a = pickle.load(open("test_file", 'rb'))
print(time.time()-start)

In [ ]:
len(a[0])